In [1]:
# Import libraries 

import numpy as np    # Numeric and matrix computation
import pandas as pd   # Optional: good package for manipulating data 
import sklearn as sk  # Package with learning algorithms implemented
import matplotlib.pyplot as  plt


df = pd.read_csv("bank-additional-full.csv", header=0) 
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
# Separation into data and label
#X = df.drop(columns=['y'])
#y = df['y']
# Codificación de la columna y a valores binarios
df['y'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Separación de características y etiquetas
X = df.drop(['y'], axis=1)  # Datos
y = df['y']  # Etiqueta

print(X.shape)
print(y.shape)


(39191, 20)
(39191,)


In [3]:
from sklearn.naive_bayes import GaussianNB       ### Because continuous data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, train_test_split
import sklearn.neighbors as nb

# Convertir características categóricas a numéricas usando One-Hot Encoding
#X_encoded = pd.get_dummies(X, drop_first=True)
# One-hot encoding para variables categóricas
X = pd.get_dummies(X)


# Usar shuffle=True para mezclar los datos
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1) 

# Crear y evaluar el modelo
gnb = GaussianNB()
cv_scores = cross_val_score(gnb,X=X,y=y,cv=cv)
np.mean(cv_scores)
print("Promedio de exactitud:", np.mean(cv_scores))

#print("Mean CV Score:", mean_score)




Promedio de exactitud: 0.8670358366617889


In [4]:
from sklearn.model_selection import cross_val_predict  
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Codificar la variable objetivo
#le = LabelEncoder()
#y_encoded = le.fit_transform(y)

# Realizar predicciones usando cross_val_predict
#predicted = cross_val_predict(GaussianNB(), X=X_encoded, y=y_encoded, cv=cv)
#conf_matrix = confusion_matrix(y_encoded, predicted)
#accuracy = accuracy_score(y_encoded, predicted)
predicted = cross_val_predict(GaussianNB(), X=X, y=y,  cv=cv)  

# Calcular y mostrar la matriz de confusión y la precisión
conf_matrix = confusion_matrix(y, predicted)
accuracy = accuracy_score(y, predicted)
print("conf_matrix Score:",conf_matrix)
print("Accuracy Score:", accuracy)

conf_matrix Score: [[31776  3055]
 [ 2156  2204]]
Accuracy Score: 0.8670357990354929


In [5]:
from sklearn.metrics import classification_report
print(classification_report(y, predicted))
#print(classification_report(y_encoded, predicted))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92     34831
           1       0.42      0.51      0.46      4360

    accuracy                           0.87     39191
   macro avg       0.68      0.71      0.69     39191
weighted avg       0.88      0.87      0.87     39191



In [6]:
## Adjusting probability threshold

In [7]:
# Focus on predict class 1

#y2 = np.zeros((y_encoded.shape))
#y2[y_encoded == 1] = 1  # Asumiendo que '1' es la clase positiva
#y2[y_encoded == 0] = 0  # Clase negativa
#(X_train, X_test, y_train, y_test) = train_test_split(X_encoded, y2, test_size=.3, random_state=1)
#y2 = np.zeros((y.shape))
y2 = y

(X_train, X_test,  y_train, y_test) = train_test_split(X, y2, test_size=.3, random_state=1)
#(X_train, X_test,  y_train, y_test) = train_test_split(X, y2, test_size=.3, random_state=1)



In [8]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

#def filterp(th,ProbClass1):
#      y=np.zeros(ProbClass1.shape[0])
#    for i,v in enumerate(ProbClass1):
 #       if ProbClass1[i]>th:
 #           y[i]=1
#    return y  

 # Ajuste de umbral de probabilidad para la clase positiva
def filterp(th, ProbClass1):
    """ Dado un umbral 'th' y un conjunto de probabilidades de pertenecer a la clase 1 'ProbClass1', retorna predicciones """
    return np.where(ProbClass1 > th, 1, 0)   

clf = GaussianNB()
lth=[]

# We do a 10 fold crossvalidation with 10 iterations
kf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_train, y_train):
    #X_train2, X_test2 = X_encoded[train_index], X_encoded[test_index]
    X_train2, X_test2 = X.iloc[train_index], X.iloc[test_index]
    y_train2, y_test2 = y2[train_index], y2[test_index]

    # Train with the training data of the iteration 
    clf.fit(X_train2, y_train2)
    # Obtaining probablity predictions for test data of the iterarion
    probs = clf.predict_proba(X_test2)
    # Collect probabilities of belonging to class 1
    ProbClass1 = probs[:,1]
    # Sort probabilities and generate pairs (threshold, f1-for-that-threshold) 
    res = np.array([[th,f1_score(y_test2,filterp(th,ProbClass1),pos_label=1)] for th in np.sort(ProbClass1)])

    # Uncomment the following lines if you want to plot at each iteration how f1-score evolves increasing the threshold 
    #plt.plot(res[:,0],res[:,1])
    #plt.show()

    # Find the threshold that has maximum value of f1-score
    maxF = np.max(res[:,1])
    pl = np.argmax(res[:,1])
    optimal_th = res[pl,0]
    
    # Store the optimal threshold found for the current iteration
    lth.append(optimal_th)

# Compute the average threshold for all 10 iterations    
thdef = np.mean(lth)
print("Selected threshold in 10-fold cross validation:", thdef)
print()


Selected threshold in 10-fold cross validation: 0.04537002370220737



In [9]:
# Train a classifier with the whole training data 
clf = GaussianNB()
clf.fit(X_train, y_train)
# Obtain probabilities for data on test set
probs = clf.predict_proba(X_test)
# Generate predictions using probabilities and threshold found on 10 folds cross-validation

In [10]:
pred = filterp(thdef,probs[:,1])
# Print results with this prediction vector
print(classification_report(y_test, pred))

# Ignore warnings explaining that in some iterations f1 score is 0


              precision    recall  f1-score   support

           0       0.95      0.87      0.91     10440
           1       0.37      0.61      0.46      1318

    accuracy                           0.84     11758
   macro avg       0.66      0.74      0.68     11758
weighted avg       0.88      0.84      0.86     11758



In [11]:
clf = GaussianNB()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92     10440
           1       0.43      0.52      0.47      1318

    accuracy                           0.87     11758
   macro avg       0.68      0.72      0.70     11758
weighted avg       0.88      0.87      0.87     11758



In [12]:
print(df['y'].unique())  # Para ver los valores únicos de la columna 'y'
print(df['y'].value_counts())  # Para ver cuántos son '1' y cuántos son '0'


[0 1]
y
0    34831
1     4360
Name: count, dtype: int64
